In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
sys.path.append('/content/drive/MyDrive/DCDP/mean')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from cpd_utils import *

import time
import bisect

import pandas as pd

# DCDP

In [ ]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[15.8113883 15.8113883 15.8113883]


In [ ]:
Y_train_list.shape

(100, 200, 20)

In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [200, 300, 500]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 2, 
                    buffer = 2, step_refine = 1, buffer_refine = 2, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 0.0, avg time: 0.6714137816429138
std loc error: 0.0, std time: 0.15578752080434435
K < K*: 0, K = K*: 100, K > K*: 0


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)

In [ ]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[3.16227766 3.16227766 3.16227766]


In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [100, 200, 300]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 1, 
                    buffer = 2, step_refine = 1, buffer_refine = 1, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 0.51, avg time: 0.658570613861084
std loc error: 0.768049477572897, std time: 0.15880409665919454
K < K*: 0, K = K*: 100, K > K*: 0


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)

In [ ]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 20
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 0.5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[1.58113883 1.58113883 1.58113883]


In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [50, 100]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 1, 
                    buffer = 2, step_refine = 1, buffer_refine = 1, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 8.3, avg time: 0.4103312563896179
std loc error: 12.904650324592295, std time: 0.02012108009246334
K < K*: 8, K = K*: 90, K > K*: 2


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)

In [ ]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[15.8113883 15.8113883 15.8113883]


In [ ]:
Y_train_list.shape

(100, 200, 100)

In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [300, 500]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 1, 
                    buffer = 2, step_refine = 1, buffer_refine = 2, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 0.0, avg time: 0.5539095139503479
std loc error: 0.0, std time: 0.02772622433974706
K < K*: 0, K = K*: 100, K > K*: 0


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)

In [ ]:
T = 4
Delta = 50
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 1

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[3.16227766 3.16227766 3.16227766]


In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [200, 300, 500]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 1, 
                    buffer = 2, step_refine = 1, buffer_refine = 1, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 0.83, avg time: 0.8498011493682861
std loc error: 0.8724104538575865, std time: 0.1665236694743707
K < K*: 0, K = K*: 100, K > K*: 0


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)

In [ ]:
T = 4
Delta = 200
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 100
theta = np.zeros((T, p))
for t in range(T):
    theta[t, 5 * t: 5 * (t + 1)] = 0.5

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5
print(diff)

nt = Delta * T

path = '/content/drive/MyDrive/DCDP/mean/'
with open(path + 'data_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '.npz', 'rb') as f:
    data = np.load(f)
    Y_train_list = data.f.Y_train_list
    Y_test_list = data.f.Y_test_list
    cp_truth_list = data.f.cp_truth_list
    theta = data.f.theta

[1.58113883 1.58113883 1.58113883]


In [ ]:
np.random.seed(0)

grid_n = 100
gamma_list = [200, 300, 500]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)
K_dc = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    Y_train = Y_train_list[b]
    Y_test = Y_test_list[b]
    cp_truth = cp_truth_list[b]

    start_time = time.time()
    dcdp = dcdp_cv_grid_mean(grid_n, lam_list, gamma_list, smooth = 1, 
                    buffer = 2, step_refine = 1, buffer_refine = 1, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    run_time_dc[b] = time.time() - start_time
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    K_dc[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_dc.std(), run_time_dc.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_dc < T - 1), sum(K_dc == T - 1), sum(K_dc > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 9.36, avg time: 2.0549121117591858
std loc error: 29.95547362336306, std time: 0.26567175631446394
K < K*: 3, K = K*: 97, K > K*: 0


In [ ]:
import pickle
with open(path + 'dcdp_n' + str(nt) + '_p' + str(p) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_kappa' + str(int(np.mean(diff) * 100)) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_dc, run_time_dc, K_dc], f)